In [ ]:
import os

import time
import json
import random

import numpy as np
import tensorflow as tf
import tensorflow_text as text

from ipywidgets import Button, HBox, VBox, RadioButtons
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
os.path.abspath('./')

In [ ]:
saved_model_path = ''.join(os.path.abspath('./')+'/swiggy_review_bert/content/swiggy_review_bert')

In [ ]:
print(tf.__version__)

In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)

In [ ]:
sentiment = [
        ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive'],
        ['Moderatly Negative', 'Neutral Negative', 'Neutral Positive', 'Moderatly Positive']
    ]

In [ ]:
def get_sentiment(pred_class):
    if(len(pred_class)==1): return sentiment[0][pred_class[0]]
    if(len(pred_class)==2): return sentiment[1][sum(pred_class)//2]
    
    return 'Mixed'

def get_response(pred, threshold=0.7):
    p_idx_sort = np.argsort(pred)
    
    for i in range(1,6):
        pred_prob = sum(pred[p_idx_sort[-i:]])

        if(pred_prob>=threshold):
            response = p_idx_sort[-i:]
            return response[::-1], pred_prob


def bot_api(query):    
    response = []
    
    result = tf.nn.softmax(reloaded_model(tf.constant([query])))
    pred = result.numpy()[0]
    threshold = 0.75
    
    pred_class, pred_prob = get_response(pred, threshold=threshold)
    sent = get_sentiment(pred_class)
    score = round(pred_prob*100,2)
    
    return [sent, score]

In [ ]:
query = 'Very poor service! Ordered items half an hour ago, a message came that item was unavailable. Then requested change in order item. One hour later, got message that new item is also unavailable. I was actually ordering food from the menu shown in swigy app only; very irresponsible service!! Really frustrating, pathetic response.'

In [ ]:
bot_api(query)

### Interface

In [ ]:
class BotButtons:
    
    def __init__(self):
        self.startBtn = Button(description='🚀 Start', button_style='success')        
        self.rateUsBtn = Button(description='⭐Rate us')
        self.feedbackBtn = Button(description='🤝Submit feedback')
        self.talkAgentBtn = Button(description='👩‍💼Talk to human agent')

        self.mainMenuBox = HBox([self.rateUsBtn, self.feedbackBtn, self.talkAgentBtn])
        
        self.ratingsBtn = RadioButtons(
            options=['⭐', '⭐⭐', '⭐⭐⭐','⭐⭐⭐⭐','⭐⭐⭐⭐⭐'],
            description='Rate us:',
            disabled=False
        )
        
        self.menuBtn = Button(description='👈Go to main menu', button_style='info')
        self.submitRatingBtn = Button(description='Submit',button_style='info')
        
        self.exitBtn = Button(description='Exit bot', button_style='danger')
        
        self.menuExitBox = HBox([self.menuBtn, self.exitBtn])
        
        self.userInputField = widgets.Textarea(
                                value='',
                                placeholder='Type something',
                                description='You: ',
                                disabled=False
                            )
        display(self.userInputField)
        self.userInputField.layout.display='none'
        
        self.submitFeedbackBtn = Button(description='Submit',button_style='info')
        
        self.yesBtn = Button(description='👍Yes', button_style='success')
        self.noBtn = Button(description='👎No', button_style='warning')
        
        self.yesNoBox = HBox([self.yesBtn, self.noBtn, self.exitBtn])
        
        self.out = widgets.Output()


In [ ]:
def goto_main_menu(obj):
    print('How may I help you today?')
    
    # botObj.rateUsBtn
    # botObj.feedbackBtn
    # botObj.talkAgentBtn
    
    display(botObj.mainMenuBox)

def rate_us_eventhandler(obj):
    botObj.mainMenuBox.layout.display = 'none'
    
    display(botObj.ratingsBtn)
    submit = botObj.submitRatingBtn
    submit.layout.display = 'visible'
    display(submit)

def talk_to_agent_eventhandler(obj):
    botObj.mainMenuBox.layout.display = 'none'
    print('agent_info')

def submit_rating_eventhandler(obj):
    botObj.ratingsBtn.layout.display = 'none'
    obj.layout.display = 'none'
    
    print('You\'ve rated us: ', botObj.ratingsBtn.index+1, '⭐')
    print('Would you like to tell us more about it?')

    display(botObj.yesNoBox)


def feedback_eventhandler(obj):
    botObj.mainMenuBox.layout.display = 'none'
    botObj.yesNoBox.layout.display = 'none'

    botObj.userInputField.layout.display = 'visible'
    display(botObj.userInputField)
    
    submit = botObj.submitFeedbackBtn
    submit.layout.display='visible'
    display(submit)

def proceed_eventhandler(obj):
    botObj.yesNoBox.layout.display = 'none'
    print('Thank you bringing that to our notice! We will surely try to improve on it.')

def submit_feedback_event(obj):
    print("User: ", botObj.userInputField.value)
    print(bot_api(botObj.userInputField.value))
    botObj.userInputField.value = ''
    
    botObj.userInputField.layout.display = 'none'
    obj.layout.display = 'none'

    # print('Continue?')

    botObj.yesNoBox.layout.display = 'visible'
    display(botObj.yesNoBox)

def exit_eventhandler(obj):
    widgets.Widget.close_all()
    print('Thank you for using feedback bot! Have a nice day :)')

def start_eventhandler(obj):
    obj.close()
    botObj.submitRatingBtn.on_click(submit_rating_eventhandler)
    botObj.rateUsBtn.on_click(rate_us_eventhandler)
    botObj.feedbackBtn.on_click(feedback_eventhandler)
    botObj.talkAgentBtn.on_click(talk_to_agent_eventhandler)
    botObj.yesBtn.on_click(feedback_eventhandler)
    botObj.noBtn.on_click(proceed_eventhandler)
    botObj.exitBtn.on_click(exit_eventhandler)
    botObj.submitFeedbackBtn.on_click(submit_feedback_event)
    
    goto_main_menu(obj)

In [ ]:
def runBot():    
    print('👋 Hello, welcome to the feedback bot!')
    start = botObj.startBtn
    display(start, botObj.out)
    start.on_click(start_eventhandler)    

In [ ]:
botObj = BotButtons()
runBot()

#### Debugging

In [ ]:
a = np.array([10, 100, 22, 38, 49, 202])

print(np.argsort(a))
print(sum(a[np.argsort(a)[-1:]]))
np.argsort(a)[-2:]+1

In [ ]:

response = []
result = tf.nn.softmax(reloaded_model(tf.constant(query)))
pred = result.numpy()[0]
threshold = 0.75
p_idx_sort = np.argsort(pred)

for i in range(1,6):
    pred_prob = sum(pred[p_idx_sort[-i:]])

    if(pred_prob>=threshold):
        response = p_idx_sort[-i:]
        print(pred[response[::-1]])
        break

In [ ]:
pred[response[::-1]]

In [ ]:
response

In [ ]:
results = tf.nn.softmax(reloaded_model(tf.constant(queries)))

In [ ]:
def print_results(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()

print_results(queries, results)